In [54]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm
nonalphanum = ''.join(c for c in map(chr, range(256)) if not c.isalnum())
import warnings
warnings.filterwarnings('ignore')

### School Meal Participation by Community Area

In [191]:
cp = "School Meal Participation Rates by Community Area\CommunitySnapshotsSchoolMealParticipation.csv"
cp_df = pd.read_csv(cp)
cp_df.rename(columns = lambda x: x.translate(None, nonalphanum).lower(), inplace=True)

cp2015_cols = [col for col in cp_df.columns if "2015" in col or col == "communityarea"]
cp2016_cols = [col for col in cp_df.columns if "2016" in col or col == "communityarea"]
cp2015_df = cp_df[cp2015_cols]
cp2016_df = cp_df[cp2016_cols]

### School Breakfast

In [227]:
brk2015 = "School Breakfast\\SchoolBreakfast2015.csv"
brk2016 = "School Breakfast\\SchoolBreakfast2016.csv"
brk2015_df = pd.read_csv(brk2015)
brk2016_df = pd.read_csv(brk2016)
brk2015_df.rename(columns = lambda x: x.translate(None, nonalphanum).lower(), inplace=True)
brk2016_df.rename(columns = lambda x: x.translate(None, nonalphanum).lower(), inplace=True)

brk2015_df["fulladdress"] = brk2015_df.apply(lambda x: x.streetaddress + ", " + x.city + " " + x.state + ", " + str(x.zip), axis = 1)
brk2016_df["fulladdress"] = brk2016_df.apply(lambda x: x.sitestreetaddress + ", " + x.sitecity + " " + x.sitestate + ", " + str(x.sitezip), axis = 1)

brk2015_df.rename(columns = lambda x: x.translate(None, nonalphanum).lower() + "2015", inplace=True)
brk2016_df.rename(columns = lambda x: x.translate(None, nonalphanum).lower() + "2016", inplace=True)

brk2015_df["chicago2015"] = brk2015_df.city2015.apply(lambda x: 1 if "chicago" in x.translate(None, nonalphanum).lower() else 0)
brk2016_df["chicago2016"] = brk2016_df.sitecity2016.apply(lambda x: 1 if "chicago" in x.translate(None, nonalphanum).lower() else 0)

brk2015_df = brk2015_df[brk2015_df.chicago2015 == 1]
brk2016_df = brk2016_df[brk2016_df.chicago2016 == 1]

In [228]:
loc2015 = brk2015_df[["entityname2015", "fulladdress2015"]]
loc2016 = brk2016_df[["siteentityname2016", "fulladdress2016"]]
loc2015.columns = ["entityname", "fulladdress"]
loc2016.columns = ["entityname", "fulladdress"]
loc = pd.concat([loc2015, loc2016])
loc.drop_duplicates(subset = ["entityname", "fulladdress"], inplace = True)
loc.to_csv("InsitutaionAddresses.csv")

### Food Insecurity Rates

In [194]:
fir = "Food Insecurity Data\\fir2014.csv"
fir_df = pd.read_csv(fir)
fir_df.rename(columns = lambda x: x.translate(None, nonalphanum).lower(), inplace=True)

### Merge

In [195]:
cp_df["id"] = cp_df.communityarea.apply(lambda x: str(x).translate(None, nonalphanum).lower())
fir_df["id"] = fir_df.chicagocommunityarea.apply(lambda x: str(x).translate(None, nonalphanum).lower())

cpfir_df = cp_df.merge(fir_df, on = "id", how = "left")

In [233]:
brk2015_df["id"] = brk2015_df["sitenumber2015"]
brk2016_df["id"] = brk2016_df["sitenumber2016"]

In [237]:
brk_df = brk2015_df.merge(brk2016_df, on = "id", how = "left")

In [243]:
brk_agg = brk_df.groupby(["id"]).mean()
brk_agg.reset_index(level=0, inplace=True)

In [245]:
brk_agg.dtypes

id                                 int64
programyear2015                  float64
sitenumber2015                   float64
claimcalendaryear2015            float64
claimcalendarmonth2015           float64
federalfiscalyear2015            float64
statefiscalyear2015              float64
mealcount2015                    float64
enrol2015                        float64
ada2015                          float64
eligibles2015                    float64
daysclaimed2015                  float64
adp2015                          float64
zip2015                          float64
zipplus42015                     float64
chicago2015                      float64
programyear2016                  float64
sitenumber2016                   float64
claimcalendaryear2016            float64
freemealcount2016                float64
reducedmealcount2016             float64
paidmealcount2016                float64
freeandreducedmealcount2016      float64
freeeligibles2016                float64
reducedeligibles

In [ ]:
brk_agg = brk_agg[["id", ]]